# Inverse Geometry of the Kuka Robot

Using the forward kinematics function developed previously, we will see how we can compute its inverse for the Kuka iiwa robot seen in class.

<img src="./figs/kuka_robot.png" width="250">

In [ ]:
#setup nice plotting
%matplotlib widget

# we import useful libraries
import time
import numpy as np
import pinocchio
import os
import meshcat
import matplotlib as mp
import matplotlib.pyplot as plt
import scipy


np.set_printoptions(suppress=True, precision=4) # we don't want to print every decimal!

In [ ]:
# the directory where the robot models are located
package_dirs = './urdf/'

# the name of the URDF model (the robot model)
urdf_file = 'iiwa.urdf'
END_EFF_FRAME_ID = 17 # number of the frame corresponding to the end-effector

# we load the urdf models with Pinocchio (it will build a robot model, its kinematic tree, etc)
urdf = package_dirs + urdf_file
robot = pinocchio.RobotWrapper.BuildFromURDF(urdf, package_dirs)

# we create the visualization for Meshcat
viz = pinocchio.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)
viz.loadViewerModel()

q = pinocchio.neutral(robot.model)
viz.display(q) # to update the meshcat display

# create a ball to visualize the goal
viz.viewer['ball'].set_object(meshcat.geometry.Sphere(0.065), 
                              meshcat.geometry.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))

## Basic transforms

We define below basic functions to get homogeneous transforms that correspond to pure translations and rotations around x,y or z axes

In [ ]:
def translate(vector):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0:3,3] = vector

    ### we return the object
    return transform

def rotateX(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[1:3,1:3] = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle), np.cos(angle)]])

    ### we return the object
    return transform

def rotateY(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0,0] = np.cos(angle)
    transform[0,2] = np.sin(angle)
    transform[2,0] = -np.sin(angle)
    transform[2,2] = np.cos(angle)

    ### we return the object
    return transform

def rotateZ(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0:2,0:2] = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle), np.cos(angle)]])

    
    ### we return the object
    return transform

## Forward kinematics
This is directly taken from the previous notebook on forward kinematics

In [ ]:
def forward_kinematics_KUKA(theta):
    T_BJ1 = translate([0,0,0.1575]) @ rotateZ(theta[0])
    T_J1J2 = translate([0,0,0.2025]) @ rotateY(theta[1])
    T_J2J3 = translate([0,0,0.2045]) @ rotateZ(theta[2])
    T_J3J4 = translate([0,0,0.2155]) @ rotateY(-theta[3])
    T_J4J5 = translate([0,0,0.1845]) @ rotateZ(theta[4])
    T_J5J6 = translate([0,-0.0607, 0.2155]) @rotateY(theta[5])
    T_J6J7 = translate([0,0.0607, 0.0810])@ rotateZ(theta[6])
    T_J7E = translate([0,0,0.04])

    T_SF = T_BJ1 @ T_J1J2 @ T_J2J3 @ T_J3J4 @ T_J4J5 @ T_J5J6 @ T_J6J7 @ T_J7E
    
    ### we return the pose of the end-effector
    return T_SF

## Inverse Geometry
We have a FK function that maps joint angles $\theta$ to the end-effector pose (as an homogeneous transform): it computes the position and the orientation of the end-effector as a function of the joint angles $p_{effector}(\theta)$ and $R_{effector}(\theta)$.

The inverse problem (denoted inverse geometry in this notebook) is to find joint angles that would put the end-effector at a given pose. For example we want to end-effector to reach a desired position $p_{desired}$.

So the problem is to find $\theta$ such that $$p_{effector}(\theta) = p_{desired}$$

This equation is complicated (there are a lot of sine and cosine) and we don't even have an explicit analytical formula for $p_{effector}(\theta)$! One way to approach the problem is to write this problem as an optimization problem (and then ask an optimization routine to find the solution for us). We can write the minimization problem
$$\min_{\theta} = ||p_{effector}(\theta) - p_{desired}||^2$$

If equation $$p_{effector}(\theta) = p_{desired}$$ has a solution, then the above minimum should be 0. In that case the minimization problem would find the solution to this equation. If the equation has no solution, then the minimization problem will find the "closest" solution (in the sense of the Euclidian distance). It means that it will find joint angles that bring the robot's end-effector closest to the desired position.

### Solving the minimization problem
We can use "off-the-shelf" optimization solvers such as the ``minimize`` function from [SciPy](https://scipy.org/) to solve such problems. The cell below 

In [ ]:
## first we need to write a function that implements the cost function we want to minimize, i.e. ||p(theta) - p_desired||^2
def get_error(theta, desired_position):
    """
    theta is the current joint configuration
    desired_position is the desired end-effector position (a 3D vector)
    """
    # we compute the end-effector position from the current guess theta
    current_guess_position = forward_kinematics_KUKA(theta)[0:3,3]
    
    # we compute and return error (as Euclidian distance)
    error = current_guess_position - desired_position
    return error.dot(error)

# our initial guess
x0 = np.zeros(7)
viz.display(x0)

# the desired position
des_pos = np.array([0.5, 0.5, 0.5])
# we display it
viz.viewer['ball'].set_transform(meshcat.transformations.translation_matrix(des_pos))

print(f"the desired position of the end-effector is: {des_pos}")

In [ ]:
res = scipy.optimize.minimize(get_error, x0, args = (des_pos))

print(f"the joint angle solution for after the optimization is:\n {res.x}")
print(f"which would lead to an end-effector position of: {forward_kinematics_KUKA(res.x)[0:3,3]}")
print(f"hence we made an error of {np.linalg.norm(forward_kinematics_KUKA(res.x)[0:3,3] - des_pos)*1000} millimeters")


#### here we display our solution ####
viz.display(res.x)

## adding bounds
It is further possible to add bounds on the joint configuration, e.g. to make sure we find a solution within the joint limits. Here is an example below where we constraint the solution to have 0 for the first joint.

In [ ]:
bounds = [(0., 0.),
          (-np.pi, np.pi),
          (-np.pi, np.pi),
          (-np.pi, np.pi),
          (-np.pi, np.pi),
          (-np.pi, np.pi),
          (-np.pi, np.pi)]

res = scipy.optimize.minimize(get_error, x0, args = (des_pos), bounds=bounds)

print(f"the joint angle solution for after the optimization is:\n {res.x}")
print(f"which would lead to an end-effector position of: {forward_kinematics_KUKA(res.x)[0:3,3]}")
print(f"hence we made an error of {np.linalg.norm(forward_kinematics_KUKA(res.x)[0:3,3] - des_pos)*1000} millimeters")


#### here we display our solution ####
viz.display(res.x)